# Data Loading and Cleaning and Creating Daily Data
## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

## Customer Data

In [5]:
customers = pd.read_csv("./Raw_Data/Customers_Orders_Data.csv")
customers = customers.drop('Unnamed: 0',axis=1)
customers.head(5)

C:\Users\gagan\AppData\Local\Temp\ipykernel_7176\115974500.py:1: DtypeWarning: Columns (5,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  customers = pd.read_csv("./Raw_Data/Customers_Orders_Data.csv")


,fsn_id,order_date,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,order_payment_type,sla,cust_id,pincode,product_mrp,product_procurement_sla
0,ACCCX3S58G7B5F6P,2023-10-17 15:11:54,3.419301e+15,3.419301e+15,6400,1,\N,\N,COD,5,-1.01299130778588E+018,-7.79175582905735E+018,7190,0
1,ACCCX3S58G7B5F6P,2023-10-19 10:07:22,1.420831e+15,1.420831e+15,6900,1,\N,\N,COD,7,-8.99032457905512E+018,7.33541149097431E+018,7190,0
2,ACCCX3S5AHMF55FV,2023-10-20 15:45:56,2.421913e+15,2.421913e+15,1990,1,\N,\N,COD,10,-1.0404429420466E+018,-7.47768776228657E+018,2099,3
3,ACCCX3S5AHMF55FV,2023-10-14 12:05:15,4.416592e+15,4.416592e+15,1690,1,\N,\N,Prepaid,4,-7.60496084352714E+018,-5.83593163877661E+018,2099,3
4,ACCCX3S5AHMF55FV,2023-10-17 21:25:03,4.419525e+15,4.419525e+15,1618,1,\N,\N,Prepaid,6,2.8945572083453E+018,5.34735360997242E+017,2099,3


In [6]:
print(customers.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1648824 entries, 0 to 1648823
Data columns (total 14 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   fsn_id                   1648824 non-null  object 
 1   order_date               1648824 non-null  object 
 2   order_id                 1648824 non-null  float64
 3   order_item_id            1648824 non-null  float64
 4   gmv                      1648824 non-null  object 
 5   units                    1648824 non-null  int64  
 6   deliverybdays            1648824 non-null  object 
 7   deliverycdays            1648824 non-null  object 
 8   order_payment_type       1648824 non-null  object 
 9   sla                      1648824 non-null  int64  
 10  cust_id                  1648824 non-null  object 
 11  pincode                  1648824 non-null  object 
 12  product_mrp              1648824 non-null  int64  
 13  product_procurement_sla  1648824 non-null 

### Wrangle Data based on data description

In [ ]:
# This ensures that no values of the wrong data type end up in any of the columns
customers['order_date'] = pd.to_datetime(customers['order_date'])
customers['order_id'] = pd.Categorical(customers.order_id)
customers['order_item_id'] = pd.Categorical(customers.order_item_id)
customers['fsn_id'] = pd.Categorical(customers.fsn_id)
customers['cust_id'] = pd.Categorical(customers.cust_id)
customers['gmv'] = pd.to_numeric(customers['gmv'], errors='coerce')
customers['deliverybdays'] = customers['deliverybdays'].replace('\\N', 0)
customers['deliverycdays'] = customers['deliverycdays'].replace('\\N', 0)
customers['deliverybdays'] = pd.to_numeric(customers['deliverybdays'], errors='coerce')
customers['deliverycdays'] = pd.to_numeric(customers['deliverycdays'], errors='coerce')
customers['order_payment_type'] = pd.Categorical(customers['order_payment_type'])

### Data Cleaning

In [8]:
print(customers.columns)

Index(['fsn_id', 'order_date', 'order_id', 'order_item_id', 'gmv', 'units',
       'deliverybdays', 'deliverycdays', 'order_payment_type', 'sla',
       'cust_id', 'pincode', 'product_mrp', 'product_procurement_sla'],
      dtype='object')


In [ ]:
#removed since pincode has errors
customers.drop(['pincode'],axis =1,inplace=True) # for now remove pincode

# remove data outside the specified timeframe
customers.drop(customers.loc[customers['order_date'].dt.strftime('%Y-%m').isin(['2023-05','2023-06','2024-07'])].index,inplace=True)
customers.drop(customers[customers['deliverycdays'] < 0].index,inplace=True)

# remove non-sensical values
customers.drop(customers[customers['gmv'] <= 0].index,inplace=True)
customers.drop(customers[customers['units'] <= 0].index,inplace=True)
customers.drop(customers[customers['product_mrp'] <= 0].index,inplace=True)
customers.drop(customers[customers['product_mrp'] < customers['gmv']/customers['units']].index,inplace=True)

# remove duplicate values from the dataset
customers.drop_duplicates(inplace=True)

In [12]:
customers.isna().sum()

fsn_id                        0
order_date                    0
order_id                      0
order_item_id                 0
gmv                        4023
units                         0
deliverybdays                 0
deliverycdays                 0
order_payment_type            0
sla                           0
cust_id                       0
product_mrp                   0
product_procurement_sla       0
dtype: int64

In [23]:
# Fill in the nan value of a GMV of a given fsn_id with the mean of the remaining data corresponding to that fsn_id
customers['temp']=customers['gmv']/customers['units']
fsn_id_gmv_modes = customers.groupby('fsn_id')['temp'].transform('mean')
customers['gmv'] = customers['gmv'].fillna(fsn_id_gmv_modes*customers['units'])
customers.drop('temp',axis=1,inplace=True)
customers.dropna(inplace=True)

C:\Users\gagan\AppData\Local\Temp\ipykernel_7176\1975208610.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fsn_id_gmv_modes = customers.groupby('fsn_id')['temp'].transform('mean')


## Holidays List

In [24]:
holidays = pd.read_excel("./Raw_Data/Canada Holiday List.xlsx")
holidays['Day'] = pd.to_datetime(holidays['Day'])
holidays.index = holidays['Day']
holidays = holidays.drop('Day',axis=1)
print(holidays.head(5))

                Occassion
Day                      
2023-07-01     Canada Day
2023-08-03  Civic Holiday
2023-09-07     Labour Day
2023-10-12   Thanksgiving
2023-10-31      Halloween


In [25]:
#merge with customers 
customers['date_only'] = customers['order_date'].dt.date.astype('datetime64[ns]')
result = pd.merge(customers.reset_index(),holidays,
                 left_on='date_only',
                 right_on='Day',how='left')
result.drop(['date_only'], axis=1,inplace=True)
result['Occassion'] = result['Occassion'].fillna('NoHoliday')
result['Occassion'] = pd.Categorical(result.Occassion)

In [26]:
# Holiday Week as per PS
result['order_date'] = pd.to_datetime(result['order_date'])

def get_week_start(date):
    return date - pd.Timedelta(days=date.dayofweek)

result['week_start'] = result['order_date'].apply(get_week_start).dt.normalize()

holidays_dt = pd.to_datetime(holidays.index)
holiday_weeks = {get_week_start(holiday).normalize() for holiday in holidays_dt}

result['Holiday_Week'] = result['week_start'].isin(holiday_weeks).astype(int)
result.drop('week_start', axis=1, inplace=True)
result.sort_values('order_date').head(5)

,index,fsn_id,order_date,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,order_payment_type,sla,cust_id,product_mrp,product_procurement_sla,Occassion,Holiday_Week
1393051,1412205,AUDDGN3Q2GSCBUSX,2023-07-01 00:36:11,165147793.0,209706706.0,305.0,1,0,0,Prepaid,2,-5.23642284503954E+018,1100,2,Canada Day,1
1409788,1430199,REME4AC4ENQGKWPB,2023-07-01 00:38:19,165148039.0,209707002.0,139.0,1,0,0,Prepaid,2,2.81921709132146E+018,700,3,Canada Day,1
1372888,1391518,ACCE5FQXQNRRGYNJ,2023-07-01 03:35:02,165165918.0,209727476.0,385.0,1,0,0,Prepaid,10,-6.36522665316929E+018,699,4,Canada Day,1
1409662,1430073,REME4AC4ENQGKWPB,2023-07-01 05:30:28,165169356.0,209731373.0,396.0,4,0,0,Prepaid,1,-2.90810866605093E+018,700,3,Canada Day,1
1409663,1430074,REME4AC4ENQGKWPB,2023-07-01 06:30:36,165171457.0,209733747.0,99.0,1,0,0,Prepaid,1,-7.28653713278933E+017,700,3,Canada Day,1


In [27]:
# read directly
sales_calendar = {
    2023: [
        ('2023-07-18', '2023-07-19'),
        ('2023-08-15', '2023-08-17'),
        ('2023-08-28', '2023-08-30'),
        ('2023-10-15', '2023-10-17'), 
        ('2023-11-07', '2023-11-14'),
        ('2023-12-25', '2024-01-03')
    ],
    2024: [
        ('2024-01-20', '2024-01-22'),
        ('2024-02-01', '2024-02-02'),
        ('2024-02-20', '2024-02-21'),
        ('2024-02-14', '2024-02-15'),
        ('2024-03-07', '2024-03-09'),
        ('2024-05-25', '2024-05-27')
    ]
}

# merge with customers
def generate_sale_days(calendar):
    sale_days = []
    for year, periods in calendar.items():
        for start, end in periods:
            start_date = datetime.strptime(start, '%Y-%m-%d')
            end_date = datetime.strptime(end, '%Y-%m-%d')
            current_date = start_date
            while current_date <= end_date:
                sale_days.append(current_date.date())
                current_date += timedelta(days=1)
    return sale_days

sale_days = generate_sale_days(sales_calendar)
sale_days_set = set(sale_days)

result['SaleDay'] = result['order_date'].map(lambda x: x.date() in sale_days_set)
result.head(5)

,index,fsn_id,order_date,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,order_payment_type,sla,cust_id,product_mrp,product_procurement_sla,Occassion,Holiday_Week,SaleDay
0,0,ACCCX3S58G7B5F6P,2023-10-17 15:11:54,3.419301e+15,3.419301e+15,6400.0,1,0,0,COD,5,-1.01299130778588E+018,7190,0,NoHoliday,0,True
1,1,ACCCX3S58G7B5F6P,2023-10-19 10:07:22,1.420831e+15,1.420831e+15,6900.0,1,0,0,COD,7,-8.99032457905512E+018,7190,0,NoHoliday,0,False
2,2,ACCCX3S5AHMF55FV,2023-10-20 15:45:56,2.421913e+15,2.421913e+15,1990.0,1,0,0,COD,10,-1.0404429420466E+018,2099,3,NoHoliday,0,False
3,3,ACCCX3S5AHMF55FV,2023-10-14 12:05:15,4.416592e+15,4.416592e+15,1690.0,1,0,0,Prepaid,4,-7.60496084352714E+018,2099,3,NoHoliday,1,False
4,4,ACCCX3S5AHMF55FV,2023-10-17 21:25:03,4.419525e+15,4.419525e+15,1618.0,1,0,0,Prepaid,6,2.8945572083453E+018,2099,3,NoHoliday,0,True


## SKU Details

In [28]:
products = pd.read_csv('./Raw_Data/SKU_details.csv')
#dropped since no unique values
products = products.drop('product_analytic_super_category',axis=1)
products = products.set_index('fsn_id')
products.head(5)

,product_analytic_category,product_analytic_sub_category,product_analytic_vertical
fsn_id,,,
ACCCX3S58G7B5F6P,CameraAccessory,CameraAccessory,CameraTripod
ACCCX3S5AHMF55FV,CameraAccessory,CameraAccessory,CameraTripod
ACCCX3S5JGAJETYR,CameraAccessory,CameraAccessory,CameraTripod
ACCCX3SG2GG9YYAH,CameraAccessory,CameraAccessory,Lens
ACCCX3SGAR6NNBMR,CameraAccessory,CameraAccessory,Lens


In [31]:
# merge with customer data
df = pd.merge(result,products,
                 left_on='fsn_id',
                 right_on='fsn_id',how='left')
df.drop('index',axis=1,inplace=True)
df.head(5)

,fsn_id,order_date,order_id,order_item_id,gmv,units,deliverybdays,deliverycdays,order_payment_type,sla,cust_id,product_mrp,product_procurement_sla,Occassion,Holiday_Week,SaleDay,product_analytic_category,product_analytic_sub_category,product_analytic_vertical
0,ACCCX3S58G7B5F6P,2023-10-17 15:11:54,3.419301e+15,3.419301e+15,6400.0,1,0,0,COD,5,-1.01299130778588E+018,7190,0,NoHoliday,0,True,CameraAccessory,CameraAccessory,CameraTripod
1,ACCCX3S58G7B5F6P,2023-10-19 10:07:22,1.420831e+15,1.420831e+15,6900.0,1,0,0,COD,7,-8.99032457905512E+018,7190,0,NoHoliday,0,False,CameraAccessory,CameraAccessory,CameraTripod
2,ACCCX3S5AHMF55FV,2023-10-20 15:45:56,2.421913e+15,2.421913e+15,1990.0,1,0,0,COD,10,-1.0404429420466E+018,2099,3,NoHoliday,0,False,CameraAccessory,CameraAccessory,CameraTripod
3,ACCCX3S5AHMF55FV,2023-10-14 12:05:15,4.416592e+15,4.416592e+15,1690.0,1,0,0,Prepaid,4,-7.60496084352714E+018,2099,3,NoHoliday,1,False,CameraAccessory,CameraAccessory,CameraTripod
4,ACCCX3S5AHMF55FV,2023-10-17 21:25:03,4.419525e+15,4.419525e+15,1618.0,1,0,0,Prepaid,6,2.8945572083453E+018,2099,3,NoHoliday,0,True,CameraAccessory,CameraAccessory,CameraTripod


## Export the Processed Data

In [ ]:
df.to_csv('./Data/daily_data.csv')